In [ ]:
https://github.com/MikeIvisonGIS/python_folium_tutorials/tree/main

# Overview

In this notebook we make map interactive showing locations of incidients. We want this map to be interactable (with java?) to highlight incidents with time periods,shout_tags, pager-codes.

# Data

In [210]:
import pandas as pd 
# Normalize cost data to range from 0 to 1 for heat map intensity

In [211]:
df = pd.read_csv("..\..\data\cleaned.csv")
# df.head()
# df.columns

In [212]:
df[['latitude', 'longitude']] = df['core_lat_long'].str.split(',', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

In [213]:
# df[['latitude', 'longitude']].head()
#convert to float
# df['latitude'] = df['latitude'].astype(float)
# df['longitude'] = df['longitude'].astype(float)

#show just these cols
feats=["core_lat_long", "latitude", "longitude"]
df[feats].head()

,core_lat_long,latitude,longitude
0,"56.0901146882635, -4.617763927445251",56.090115,-4.617764
1,"56.30200229959834, -4.720781936098027",56.302002,-4.720782
2,"56.14545006003698, -4.661915903592648",56.145450,-4.661916
3,"56.01681400723627, -4.611093212845342",56.016814,-4.611093
4,"56.027043486219625, -4.595378734373894",56.027043,-4.595379


In [214]:
# determine if there are NaN in df[feats].
# df[feats].isnull().sum()
#drop those with nan
df = df.dropna(subset=feats)
df[feats].isnull().sum()


core_lat_long    0
latitude         0
longitude        0
dtype: int64

In [215]:
# df.columns.to_list()

In [216]:
feats=[ "latitude", "longitude","date_of_shout","pager_code","shout_details_tags","crew_on_board","crew_on_shore","shout_details"]
df_filtered = df[feats]

# Map creation

In [217]:
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

In [218]:
# Generate a map showing all incidnets but clustered. Put a marker on the poisition of the rescue boat.

In [219]:
# for df_filtered only take rows where the date_of_shout has year 2023
df_filtered['date'] = pd.to_datetime(df_filtered['date_of_shout'], dayfirst=True)
df_filtered['year'] = df_filtered['date'].dt.year
df_filtered_2023 = df_filtered.loc[df_filtered['year'] == 2023]
df_filtered_2022 = df_filtered.loc[df_filtered['year'] == 2022]


C:\Users\RhysL\AppData\Local\Temp\ipykernel_15120\2857013651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date_of_shout'], dayfirst=True)
C:\Users\RhysL\AppData\Local\Temp\ipykernel_15120\2857013651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['year'] = df_filtered['date'].dt.year


In [220]:
def make_map(df_filtered):

    # Create a map centered around Loch lomond
    map_center = df_filtered[['latitude', 'longitude']].dropna().mean().tolist()
    # map_center
    LLRB_pos=[56.10063229251386, -4.635696317637836]

    # fix the map specifications

    min_long= -4.60
    # max_long=-4.723302633762251
    max_long=-4.65

    min_lat= 56.05
    max_lat= 56.1

    m = folium.Map(
        location=map_center,
        control_scale=True,
        zoom_control=True,
        # max_bounds=True,
        zoom_start=10.45,
        min_lat=min_lat,
        max_lat=max_lat,
        min_lon=min_long,
        max_lon=max_long,
    )

    # Note the location of LLRB

    folium.Marker(
        location = LLRB_pos,
        tooltip="LLRB position",
        # popup="Add a single point tutorial",
        icon=folium.Icon(color="blue"),
    ).add_to(m)

    # Use the Marker Cluster plugin to make the map more navigable
    # Use marker clusters: (so when zoomed out, markers are clustered together)

    # create marker_cluster variable
    marker_cluster = MarkerCluster().add_to(m)

    color_mapping={222:"yellow", 333:"orange", 999:"red"}

    # Iterate through each row of df_filtered
    for index, row in df_filtered.iterrows():
        lat = row["latitude"]
        lng = row["longitude"]
        descrip= f"{row['date_of_shout']}\n Crew on board: {row['crew_on_board']}\n Crew on shore: {row['crew_on_shore']}\n Shout tags: {row['shout_details_tags']}"
        details= f"Shout details: \n {row['shout_details']}"
        folium.Marker(
            location=[lat, lng],
            tooltip=descrip,
            popup=folium.Popup(details, max_width=300),  # Set the max_width parameter to increase the width
            icon=folium.Icon(color=color_mapping[row['pager_code']]),
        ).add_to(marker_cluster)
    return m

In [227]:
m=make_map(df_filtered_2022)
m

C:\Users\RhysL\AppData\Local\Temp\ipykernel_15120\2496405654.py:56: UserWarning: color argument of Icon should be one of: {'beige', 'orange', 'red', 'cadetblue', 'darkblue', 'pink', 'darkgreen', 'darkpurple', 'green', 'lightred', 'black', 'lightblue', 'lightgray', 'blue', 'darkred', 'purple', 'white', 'lightgreen', 'gray'}.
  icon=folium.Icon(color=color_mapping[row['pager_code']]),


In [225]:
# Save to an HTML file
m.save('2022_incident_map.html')

In [226]:
m=make_map(df_filtered_2023)
m

C:\Users\RhysL\AppData\Local\Temp\ipykernel_15120\2496405654.py:56: UserWarning: color argument of Icon should be one of: {'beige', 'orange', 'red', 'cadetblue', 'darkblue', 'pink', 'darkgreen', 'darkpurple', 'green', 'lightred', 'black', 'lightblue', 'lightgray', 'blue', 'darkred', 'purple', 'white', 'lightgreen', 'gray'}.
  icon=folium.Icon(color=color_mapping[row['pager_code']]),


In [ ]:
# Save to an HTML file
m.save('2023_incident_map.html')